In [42]:
import os
import csv
import wfdb
import heartpy as hp
import pandas as pd
import numpy as np
import scipy.signal as ssignal
from scipy.fft import fft, fftfreq
# from scipy.signal import resample, butter, filtfilt, medfilt
# from scipy.ndimage import gaussian_filter1d

In [43]:
# def funcBPFilter(ppg_signal, order, f1, f2, fs):
#     fc = [f1, f2]
#     b, a = butter(order, fc, btype='bandpass', analog=False, output='ba', fs=fs)
#     signalBP = filtfilt(b, a, ppg_signal)
#     return signalBP

# def funcMedianFilter(ppg_signal, window_size):
#     return medfilt(ppg_signal, kernel_size=window_size)

def cheby(ppg, fs, order=4, rs=20, btype='low'):
    nyquist = 0.5 * fs
    cutoff = 10 / nyquist
    rs, order = 20, 4
    b, a = ssignal.cheby2(N=order, rs=rs, Wn=cutoff, btype='low')
    return ssignal.filtfilt(b, a, ppg)

In [44]:
subjects = pd.read_csv(r"D:\Proga\AML\datasets\Autonomic_Aging\subject-info.csv",dtype = {"ID":str})
subjects.dropna(inplace=True)
subjects = subjects.reset_index()
subjects = subjects[["ID","Age_group"]]
subjects

,ID,Age_group
0,0001,2.0
1,0002,2.0
2,0003,7.0
3,0004,4.0
4,0005,2.0
...,...,...
1091,1117,3.0
1092,1118,8.0
1093,1119,2.0
1094,1120,3.0


In [45]:
ids = subjects["ID"].values
age_groups = subjects["Age_group"].values

print(len(ids))
print(len(age_groups))
print(ids[len(ids)-1])
print(age_groups[len(age_groups)-1])

1096
1096
1121
3.0


In [46]:
result = []
result_metadata = []

new_id = 0

for id in ids:
    new_id += 1 
    try:
        record = wfdb.rdrecord('D:\\Proga\\AML\\datasets\\Autonomic_Aging\\' + id)

        channel_index = record.sig_name.index('NIBP')
        signal = record.p_signal[:, channel_index].astype(np.float32)
        fs = record.fs
        five_minute_samples = 60 * 5 * fs

        signal = signal[:five_minute_samples]

        #signal = funcBPFilter(signal, order=2, f1=0.5, f2=4.0, fs=fs)
        #signal = funcMedianFilter(signal,window_size = 5)

        try:
            wd, m = hp.process(signal, fs)
            
            # Вычисляем частотные характеристики с помощью scipy
            n = len(signal)
            yf = fft(signal)
            xf = fftfreq(n, 1/fs)
            
            # Определяем границы для LF и HF
            lf_band = (0.04, 0.15)
            hf_band = (0.15, 0.4)
            
            # Вычисляем LF и HF
            lf_mask = (xf >= lf_band[0]) & (xf <= lf_band[1])
            hf_mask = (xf >= hf_band[0]) & (xf <= hf_band[1])
            
            lf = np.trapz(np.abs(yf[lf_mask])**2, xf[lf_mask])
            hf = np.trapz(np.abs(yf[hf_mask])**2, xf[hf_mask])
            lf_hf_ratio = lf / hf
            
            freq_features = {
                'lf': lf,
                'hf': hf,
                'lf_hf_ratio': lf_hf_ratio
            }

            idx = int(subjects[subjects["ID"] == id].index.to_list()[0])
            age_group = age_groups[idx]

            result_metadata.append({
                'id': new_id,
                'age_group': age_group,
                'lf': freq_features['lf'],
                'hf': freq_features['hf'],
                'lf_hf_ratio': freq_features['lf_hf_ratio'],
                **m
            })

            print((new_id, id, age_group, freq_features['lf'], freq_features['hf'], freq_features['lf_hf_ratio']))

        except hp.exceptions.BadSignalWarning as e:
            result_metadata.append({
                'id': new_id,
                'age_group': None,
                'error': "BadSignalWarning"
            })
            print((new_id, id, None, None, None, "BadSignalWarning"))

    except FileNotFoundError as e:
        result_metadata.append({
            'id': new_id,
            'age_group': None,
            'error': "FileNotFound"
        })
        print((new_id, id, None, None, None, "FileNotFound"))

    print(f"Process end: {id}")

(1, '0001', 2.0, 477260055.25333333, 112119563.74333322, 4.256706317069592)
Process end: 0001
(2, '0002', 2.0, 382848592.4266667, 41892162.25999998, 9.13890741782558)
Process end: 0002
(3, '0003', 7.0, 32090860.770000003, 68832027.56666665, 0.466219896528817)
Process end: 0003
(4, '0004', 4.0, 399697747.3066667, 36498139.24166665, 10.951181501613858)
Process end: 0004
(5, '0005', 2.0, 266344434.32000005, 182272637.1200001, 1.4612420082815332)
Process end: 0005
(6, '0006', 4.0, 205284991.44, 103120544.54666676, 1.9907283494522197)
Process end: 0006
(7, '0007', 2.0, 298598949.84000003, 56096617.53916664, 5.322940365014314)
Process end: 0007
(8, '0008', 3.0, 302806605.28, 154206106.23999998, 1.9636486042175552)
Process end: 0008
(9, '0009', 3.0, 467300010.4533332, 206813466.81333327, 2.2595240902523583)
Process end: 0009
(10, '0010', 2.0, 751392581.1200001, 62078493.85333336, 12.103911265876393)
Process end: 0010
(11, '0011', 2.0, 62038628.98666667, 19545715.773333333, 3.1740269686775755)

c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:5317: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


(33, '0033', 2.0, 735.4112106323244, 833.1166779836014, 0.8827229487377994)
Process end: 0033
(34, '0035', 12.0, 37525775.010000005, 36097168.555, 1.039576690144638)
Process end: 0035
(35, '0036', 3.0, 192706940.48000002, 300910956.0000001, 0.6404118448914168)
Process end: 0036
(36, '0037', 5.0, 848925279.8933334, 22333637.283333324, 38.01106237750407)
Process end: 0037
(37, '0038', 7.0, 820073406.7733334, 45043289.91666668, 18.2063390194306)
Process end: 0038
(38, '0039', 3.0, 277408238.82666665, 181174877.25999993, 1.5311628357201226)
Process end: 0039
(39, '0040', 2.0, 2317280063.5733333, 475325715.22666675, 4.875141380617921)
Process end: 0040
(40, '0041', 2.0, 801372833.7066668, 110155320382.93335, 0.007274935345118615)
Process end: 0041
(41, '0042', 2.0, 655585021.2266667, 362721929.5733333, 1.8074038754641213)
Process end: 0042
(42, '0043', 2.0, 82607961.77333334, 23791838.040000007, 3.4721134884345117)
Process end: 0043
(43, '0044', 1.0, 606463476.8000001, 157707564.93333337, 3

C:\Users\toha2\AppData\Local\Temp\ipykernel_3744\488822791.py:12: RuntimeWarning: overflow encountered in cast
  signal = record.p_signal[:, channel_index].astype(np.float32)


(63, '0066', 2.0, 510837888.8533334, 24638170.668333326, 20.733596488553328)
Process end: 0066
(64, '0067', 2.0, 1027914804.2666667, 247383363.17333338, 4.1551492836098305)
Process end: 0067
(65, '0068', 8.0, 224299023.02666664, 59724812.45999994, 3.7555416884211823)
Process end: 0068
(66, '0069', None, None, None, 'BadSignalWarning')
Process end: 0069
(67, '0070', 6.0, 881159064.3199999, 325331517.0666666, 2.708495851446922)
Process end: 0070
(68, '0071', 2.0, 480881856.32000005, 16289464.44083332, 29.52103539478917)
Process end: 0071
(69, '0072', 3.0, 597216397.44, 54128723.92333333, 11.033262086242482)
Process end: 0072
(70, '0073', 3.0, 8344076178.7733345, 1965487296.2399998, 4.245296418214277)
Process end: 0073
(71, '0074', 3.0, 116530441.02666666, 17359365.539999988, 6.712828343763694)
Process end: 0074
(72, '0075', 4.0, 455120135.9466666, 1248032998.4, 0.3646699538635104)
Process end: 0075
(73, '0076', 8.0, 292668253.6533334, 28647038.874166675, 10.216352724583196)
Process end: 

...

(102, '0105', 2.0, 530066525.97333324, 1068349579.1733334, 0.4961545699147349)
Process end: 0105
(103, '0106', 3.0, 360873048.42666674, 72031908.63000001, 5.009905405676971)
Process end: 0106
(104, '0107', 1.0, 566652146.3466667, 218239312.2933334, 2.5964714624147773)
Process end: 0107
(105, '0108', 13.0, 1599656248.3200004, 563639952.7199999, 2.838081723271068)
Process end: 0108
(106, '0109', 2.0, 8577659003.7333355, 1915997731.626667, 4.476862817812926)
Process end: 0109
(107, '0110', 2.0, 300918039.5733333, 83302095.80333328, 3.6123705732898532)
Process end: 0110
(108, '0111', 7.0, 86430929.92666668, 180876453.36000013, 0.4778451164930925)
Process end: 0111
(109, '0112', 4.0, 73894712.58666667, 23048823.538333327, 3.206007997057625)
Process end: 0112
(110, '0113', 3.0, 591953050.24, 51288422.53999999, 11.541650550440192)
Process end: 0113
(111, '0114', 7.0, 571495564.5866668, 107027330070.61339, 0.005339716166044798)
Process end: 0114
(112, '0115', 2.0, 153579183.92000002, 15384100.

c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


(619, '0631', 2.0, 721630802.9866667, 37774946.440000005, 19.1034235914238)
Process end: 0631
(620, '0632', 2.0, 299930487.3066666, 166586447.59999996, 1.8004495061137655)
Process end: 0632
(621, '0633', 2.0, 1061777369.5999999, 161871554683.30682, 0.0065593820463225394)
Process end: 0633
(622, '0634', 3.0, 541231066.1333334, 16134099.127500003, 33.54578782839038)
Process end: 0634
(623, '0635', 4.0, 420862855.04, 98720317.23999998, 4.263183778237219)
Process end: 0635
(624, '0636', 7.0, 426941485.97333336, 95252495.12, 4.4822078984436935)
Process end: 0636
(625, '0637', 13.0, 198940573.86666667, 184841404.64000002, 1.0762771158016593)
Process end: 0637
(626, '0638', 8.0, 267927130.45333332, 138536367.51999995, 1.9339840884355057)
Process end: 0638
(627, '0639', 2.0, 919986345.3866668, 84118877.37333336, 10.93674064744845)
Process end: 0639
(628, '0640', 11.0, 293063090.1333333, 171659117.45333332, 1.7072387093741437)
Process end: 0640
(629, '0641', 2.0, 191497961.54666674, 15596777.23

c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\heartpy\analysis.py:677: UserWarning: 
A theoretically impossible result was found during the iteration
process for finding a smoothing spline with fp = s: s too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  interp = UnivariateSpline(x, rrlist, k=3)


(891, '0908', 5.0, 317539432.85333335, 257021212.6533334, 1.2354600212770224)
Process end: 0908
(892, '0909', 7.0, 9642417909.76, 1856882749.7066674, 5.192798474369594)
Process end: 0909
(893, '0910', 3.0, 145450449.92000002, 242714198.94499987, 0.5992663410390743)
Process end: 0910
(894, '0911', 2.0, 1115010466.5600002, 173275297.31999993, 6.434907247631671)
Process end: 0911
(895, '0912', 15.0, 160841789.7066667, 530640880.1866668, 0.30310855366078543)
Process end: 0912
(896, '0914', 8.0, 1770374446.0800004, 149896652093.8668, 0.011810633668931939)
Process end: 0914
(897, '0915', 8.0, 970411762.0800002, 306198396.41333354, 3.1692254872884864)
Process end: 0915
(898, '0916', 2.0, 385406403.62666667, 161084779.5733333, 2.392568712248892)
Process end: 0916
(899, '0918', None, None, None, 'BadSignalWarning')
Process end: 0918
(900, '0919', 6.0, 272817173.70666665, 106356002.30333345, 2.565131894752647)
Process end: 0919
(901, '0920', 14.0, 86845739.86666666, 42174024.000833355, 2.0592234

In [47]:
df_metadata = pd.DataFrame(result_metadata)

metadata_path = 'D:\\Proga\\AML\\PPG_dataset\\dataset.csv'

#df_metadata.to_csv(metadata_path, index=False)

In [48]:
df1 = df_metadata.drop(columns=["error"])
df1

,id,age_group,lf,hf,lf_hf_ratio,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,1,2.0,4.772601e+08,1.121196e+08,4.256706,69.982942,857.351779,56.187352,32.28238,55.780042,0.706522,0.331522,40.0,39.440929,67.507515,8364.676182,0.584245,0.166667
1,2,2.0,3.828486e+08,4.189216e+07,9.138907,65.715434,913.027523,73.718299,46.512425,78.898203,0.779141,0.503067,50.0,55.787767,87.910442,15407.398553,0.634598,0.133333
2,3,7.0,3.209086e+07,6.883203e+07,0.466220,92.792823,646.601732,8.369797,9.726426,12.848187,0.049892,0.006508,10.0,9.085028,7.580096,216.346967,1.198537,0.333333
3,4,4.0,3.996977e+08,3.649814e+07,10.951182,62.728278,956.506410,54.304814,22.233558,36.980050,0.517685,0.131833,30.0,26.147648,72.162114,5927.775575,0.362346,0.200000
4,5,2.0,2.663444e+08,1.822726e+08,1.461242,65.202127,920.215385,79.770535,79.733137,117.900879,0.873457,0.629630,50.0,83.368465,76.207123,19959.389060,1.093972,0.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,1092,3.0,4.193181e+08,5.253909e+07,7.981070,76.421623,785.118110,45.940212,24.673881,36.173776,0.413158,0.100000,30.0,25.578391,59.756635,4801.857091,0.428043,0.133333
1092,1093,8.0,7.643899e+07,1.179163e+08,0.648248,90.942550,659.757174,12.941820,11.019298,17.218019,0.117257,0.004425,10.0,12.174938,13.676611,523.112526,0.890201,0.200000
1093,1094,2.0,6.772962e+08,3.406916e+07,19.880038,76.318969,786.174142,48.865615,20.614626,29.531532,0.326260,0.039788,30.0,20.881736,66.050257,4333.022962,0.316149,0.133333
1094,1095,3.0,7.203550e+09,1.517587e+09,4.746712,54.413543,1102.666667,77.898669,49.370235,77.636506,0.786458,0.463542,51.0,54.896623,92.130498,15889.086308,0.595857,0.200000


In [49]:
df_copy = df1.dropna()
df_copy
df_copy.to_csv(metadata_path, index=False)